In [4]:
import numpy as np
import operator

Importing data by-line:

In [251]:
with open('coursera_sessions_train.txt') as file:
    train_data = [i.replace('\n', '').split(';') for i in file.readlines()]
train_data[0:4]

[['0,1,2,3,4,5', ''],
 ['9,10,11,9,11,12,9,11', ''],
 ['16,17,18,19,20,21', ''],
 ['24,25,26,27,24', '']]

Creating bag of words from train data: 'bagow_views' - viewed items indices, 'bagow_purchases' - bought items indices (the latter are unique in each session)

In [252]:
bagow_views = []
for i in train_data:
    z = i[0].split(',')
    for j in z:
        bagow_views.append(j)
bagow_purchases = []
for i in train_data:
    z = i[1].split(',')
    for j in z:
        bagow_purchases.append(j)

Count unique indeces and create a dictionary 'item' : frequency

In [253]:
counts_views = np.unique(bagow_views, return_counts=True)
dictfreq_views = {}
for i, j in zip(counts_views[0], counts_views[1]):
    dictfreq_views[i] = j
counts_purchases = np.unique(bagow_purchases, return_counts=True)
dictfreq_purchases = {}
for i, j in zip(counts_purchases[0], counts_purchases[1]):
    dictfreq_purchases[i] = j

Define function sorts a single line in massive:

In [27]:
def string_sorter(string, dict):
    import operator
    """Sorts indices of items watched by customer popularity-wise
    """
    z = []
    for i in string:
        try:
            z.append((i, dict[i]))
        except KeyError:
            z.append((i, 0))
    return [i[0] for i in sorted(z, key=operator.itemgetter(1), reverse=True)]

In [29]:
def string_sorter_tester(string_sorter):
    base = {'1':1, '2':2, '3':3, '4':4}
    string = (['1']+['2']+['3']+['4'], ['6']+['2']+['8'])
    ans1 = ['4', '3', '2', '1']
    if string_sorter(string[0], base) != ans1:
        print('test 1 error')
    ans2 = ['2', '6', '8']
    if string_sorter(string[1], base) != ans2:
        print('test 2 error')

In [30]:
string_sorter_tester(string_sorter)

Get sorted by-line massive where each line contains tuple representing one session and consisted of two lists: first with indices of watched items and latter with indices of items that have been bought.

В итоге будем получать отсортированный массив, где каждая строка это тьюпл, представляющий одну сессию и включающий в свою очередь два листа: первый с индексами просмотренных товаров, второй с индексами купленных.

Define function sorting whole massive:

In [44]:
def massive_sorter(data, dict):
    ''' Sorts indices in massive by-line according to theirs frequencies. Massive must consist
    of lines with a tuple each. Each tuple must contain two lists, both with items indices.
    '''
    sorted = []
    for i in data:
        if len(i[1]) > 1:
            sorted.append((string_sorter(i[0], dict), string_sorter(i[1], dict)))
        else:
               sorted.append((string_sorter(i[0], dict), ''))
    return sorted

Process train data. Making each line in massive a tuple of 2 lists: viewed items indices and purchased items indices. Then soring.

In [169]:
train_data_prepared = [(i[0].split(','), i[1].split(',')) for i in train_data]

In [175]:
train_sorted_views = massive_sorter(train_data_prepared, dictfreq_views)
train_sorted_purchases = massive_sorter(train_data_prepared, dictfreq_purchases)

Import and process test data.

In [171]:
with open('coursera_sessions_test.txt') as file:
    test_data = [i.replace('\n', '').split(';') for i in file.readlines()]
test_data[0:4]

[['6,7,8', ''], ['13,14,15', ''], ['22,23', ''], ['28,29,30,31,32,33', '']]

In [172]:
test_data_prepared = [(set(i[0].split(',')), i[1].split(',')) for i in test_data]

In [173]:
test_sorted_views = massive_sorter(test_data_prepared, dictfreq_views)
test_sorted_purchases = massive_sorter(test_data_prepared, dictfreq_purchases)

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Metrics. Average precision and average recall are used.

Delete sessions without purchases.

In [176]:
train_sorted_views_with_purchases = [i for i in train_sorted_views if len(i[1]) > 0]
train_sorted_purchases_with_purchases = [i for i in train_sorted_purchases if len(i[1]) > 0]

In [174]:
test_sorted_views_with_purchases = [i for i in test_sorted_views if len(i[1]) > 0]
test_sorted_purchases_with_purchases = [i for i in test_sorted_purchases if len(i[1]) > 0]

Define function counts presicion for one session and average precision for whole massive.

In [210]:
def presicionatk(line, km):
    '''Count presicion for a line
    '''
    l = [1 if i in line[1] else 0 for i in line[0]]
    p = 0
    k = 1
    for i in l[0:km]:
        p = p + i/k
        k += 1
    return p
def presicion(massive, km):
    '''Count average presicion for a massive
    '''
    l = []
    for line in massive:
        l.append(precisionatk(line, km))
    return sum(l)/len(l)

In [245]:
def recallatk(line, km):
    '''Count presicion for a line
    '''
    l = [1 if i in line[1] else 0 for i in line[0]]
    p = 0
    b = 1
    for i in l[0:km]:
        p = p + i/b
        b = b+1 if b<len(line[1]) else b
    return p
def recall(massive, km):
    '''Count average presicion for a massive
    '''
    l = []
    for line in massive:
        l.append(presicionatk(line, km))
    return sum(l)/len(l)

In [246]:
def recalltest(recall):
    x1 = [([1, 2, 3], [0, 2]), ([4, 5, 6], [1])]
    if recall(x1, 3) != 0.25:
        print('test1 error')
    x2 = [([1, 2, 3], [0, 2]), ([4, 5], [1, 2, 3])]
    if recall(x2, 3) != 0.25:
        print('test2 error')

In [247]:
recalltest(recall)

In [248]:
k = 1
r1 = recall(train_sorted_views_with_purchases, k)
print('recall of recomendations on most viewed items on train sample:', r1)
r2 = recall(train_sorted_purchases_with_purchases, k)
print('recall of recomendations on most purchased items on train sample:', r2)
r3 = recall(test_sorted_views_with_purchases, k)
print('recall of recomendations on most viewed items on test sample:', r3)
r4 = recall(test_sorted_purchases_with_purchases, k)
print('recall of recomendations on most purchased items on test sample:', r4)

recall of recomendations on most viewed items on train sample: 0.4536637931034483
recall of recomendations on most purchased items on train sample: 0.7489224137931034
recall of recomendations on most viewed items on test sample: 0.4234913793103448
recall of recomendations on most purchased items on test sample: 0.4234913793103448


In [249]:
k = 5
r5 = recall(train_sorted_views_with_purchases, k)
print('recall of recomendations on most viewed items on train sample:', r5)
r6 = recall(train_sorted_purchases_with_purchases, k)
print('recall of recomendations on most purchased items on train sample:', r6)
r7 = recall(test_sorted_views_with_purchases, k)
print('recall of recomendations on most viewed items on test sample:', r7)
r8 = recall(test_sorted_purchases_with_purchases, k)
print('recall of recomendations on most purchased items on test sample:', r8)

recall of recomendations on most viewed items on train sample: 1.0010416666666713
recall of recomendations on most purchased items on train sample: 1.5355244252873494
recall of recomendations on most viewed items on test sample: 0.7799389367816115
recall of recomendations on most purchased items on test sample: 0.7785380747126458


In [215]:
(' ').join(list(map(lambda x: str(round(x, 2)), [p1, p2, p3, p4])))

'1.0 1.54 0.78 0.78'

In [ ]:
with open('answer1.txt', 'w') as f:
    f.write()